In [13]:
# -*- coding: utf-8 -*-

#####################################################################################################################
#####################################################################################################################
##### @author : Murat BAYRAV (data analyst)                                                                     #####
##### Date    : Dec 1, 2018                                                                                     #####
##### Project : Learning Python and Pandas especially.                                                          #####
##### Aim     : Assign works to personnel.                                                                      #####
#####           - Each workload (or each of the workloads) is indivisible...                                    #####
#####           - Each workload has to be dealt out to one of the staff completely...                           #####
#####           - Each Workload is expected to deal out to the same worker who was responsible for it before... #####
#####           - Workloads are fairly divided among staff as much as possible...                               #####
##### Input  :  Three Excel files...                                                                            ##### 
##### Output :  One Excel file...                                                                               #####
#####################################################################################################################
#####################################################################################################################

############################################ ****** Pay Attention ****** ############################################
#####################################################################################################################
##### P.S    : do not forget to set the following option in Jupyter,                                            #####
#####          if you would like to display some long text output in just one cell.                             #####
#####          Cell -> All Output -> Toggle Scrolling                                                           #####
#####################################################################################################################
#####################################################################################################################

### Import Modules...
import sys
# for importing the module that i had written before, define the folder holding this module first...
# in fact, this is an ugly hack... not recommended !
# for my home computer, this folder is below:
# pathOfProg = "C:\\Users\\Aslan Yavrusu\\OneDrive\\Desktop\\My New Way\\Programming - All\\Pandas - All\\Trainings\\Assign_Pers\\Never_Ever"
# for my office computer, this folder is below:
pathOfProg = "D:\\My_Way\\Programming_All\\Pandas_All\\Pandas_Trainings\\Assign_Pers\\Never_Ever"
# for my home computer, this folder is below:
#pathOfProg = "C:\\Users\\Aslan Yavrusu\\OneDrive\\Desktop\\My Way\\Programming - All\\Pandas - All\\Pandas_Trainings\\Assign_Pers\\Never_Ever"
sys.path.append(pathOfProg)
# Pay attention: reload the module !!!
# Because Python load the module just one time...
import mymodule ### mymodule.py file ###
### other modules...
import os
import sys
import time
import glob
import math
import xlrd # for writing to excel
import argparse # for command line...
import openpyxl # for writing to excel
import pandas as pd
import numpy as np
from datetime import datetime
from os.path import dirname, abspath, join

### define globals...
allSequences = list()
logTxtFile = None
PastAssignmentsAvailable = False

### define functions...

def setRelatedOption():
    """
        for jupyter notebook settings...
    """
    # environment settings: 
    pd.set_option('display.max_column',None) # or 1000
    pd.set_option('display.max_rows',None)
    pd.set_option('display.max_seq_items',None)
    pd.set_option('display.max_colwidth', 500)
    pd.set_option('expand_frame_repr', True)
    
def headOfProg():
    """
        displaying the main purpose of the prog...
    """
    os.system('mode con: cols=100 lines=100')
    _ = os.system('cls') #clear the screen
    print("----------------------------------------")
    print("Developer: Murat BAYRAV / 12-2018 ( @ ) ")
    print("----------------------------------------")
    print("İşin personele dağıtılması...")
    print("Küme iş, bütün olarak bir personele,")
    print("Tüm işler eksiksiz olarak,")
    print("Her küme iş geçmişte o işi yapana,")
    print("Eşit iş yükü olacak şekilde...")
    print("----------------------------------------")
    #time.sleep(2)
    
def headOfProgTurkish():
    """
        displaying the main purpose of the prog in Turkish...
    """
    _ = os.system('cls') #clear the screen
    print(" -----------------------------------------------------------------")
    print("|            @Developer : Murat BAYRAV  /  Aralık 2018            |")
    print(" -----------------------------------------------------------------")
    print("|                 İşin personele dağıtılması...                   |")
    print("|Her alt birim iş, bölünmeden, bütün olarak bir personele verilir.|")
    print("|Tüm alt birim iş eksiksiz dağıtılır.                             |")
    print("|Her alt birim iş, geçmişte o işi yapan varsa ona verilir.        |")
    print("|İş, eşit olacak şekilde personel arasında dağıtılır.             |")
    print(" -----------------------------------------------------------------\n")
    #time.sleep(2)

def allArgs():
    """
        read and check command-line arguments of the prog...
    """
    allArgsAre = list()
    
    parser = argparse.ArgumentParser()
    class CapitalisedHelpFormatter(argparse.HelpFormatter):
        def add_usage(self, usage, actions, groups, prefix=None):
            if prefix is None:
                prefix = 'Programın Kullanımı : '
            return super(CapitalisedHelpFormatter, self).add_usage(usage, actions, groups, prefix)

    parser = argparse.ArgumentParser(add_help=False, formatter_class=CapitalisedHelpFormatter)
    parser._positionals.title = 'Pozisyonel parametreler'
    parser._optionals.title = 'Opsiyonel parametreler'

    """    
    parser.prog = 'Personel-İş tahsisi'
    parser.description = "Bu program ideal personelin ilgili kayıtlara tahsisini gerçekleştirir."
    parser.epilog = "Bu işlemi, özetle şu algoritma ile yapar:"
    parser.add_argument('-a', action="store_true", default=False)
    parser.add_argument('-b', action="store", dest="b")
    parser.add_argument('-c', action="store", dest="c", type=int)
    ### parser.add_argument('-v', '--Versiyon', action='version', version='%(prog)s 1.0', help="Programın versiyonuna ilişkin bilgiler...")
    ### _VersionAction(option_strings=['-v', '--version'], dest='version', nargs=0, const=None, default='==SUPPRESS==', type=None, choices=None, help="Bu yardım dosyası yardımlarınıza hazırdır !", metavar=None)
    ### parser.add_argument('-h', '--Yardım', action='help', default=argparse.SUPPRESS, help='Programın kullanımına ilişkin bilgiler...')
    ### _HelpAction(option_strings=['-h', '--help'], dest='help', nargs=0, const=None, default='==SUPPRESS==', type=None, choices=None, help='Mesajını ver (YARDIM)', metavar=None)
    ### print(parser.format_help())
    """
    #define compulsory command-line arguments...
    parser.add_argument(dest = "Girdi", action="store", help="Tasnif edilecek girdi dosyası (csv | xlsx)")
    parser.add_argument(dest = "Personel", action="store", help="Aktif personel dosyası (xlsx)")
    #define optional command-line arguments...
    parser.add_argument("-g", "--Tarihsel", dest = "Tarihsel", action="store", help="Geçmiş atamalar dosyası (xlsx)")
    ### parser.add_argument("-v", "--verbosity", action="count", default=0)

    #get command-line values
    try:
        args = parser.parse_args()
    except:
        print("\nZorunlu argümanlar verilmelidir !")
        print("Program sonlandırıldı...\n")
        #parser.print_usage()
        print(parser.format_help())
        sys.exit(-111)

    global PastAssignmentsAvailable
    PastAssignmentsAvailable = False

    allArgsAre.append(args.Girdi)
    allArgsAre.append(args.Personel)
    if args.Tarihsel is None:
        pass # do nothing...
    else:
        allArgsAre.append(args.Tarihsel)
        # global PastAssignmentsAvailable
        PastAssignmentsAvailable = True

    print("Okunan Komut Satırı")
    print("Girdi    : {0}".format(allArgsAre[0]))
    print("Personel : {0}".format(allArgsAre[1]))
    print("Tarihsel : {0}".format(allArgsAre[2]))
    print('Klasör   : ' + os.getcwd())
    # write into the file...
    logTxtFile.filePrint("\nOkunan Komut Satırı\n")
    logTxtFile.filePrint("Girdi    : {0}\n".format(allArgsAre[0]))
    logTxtFile.filePrint("Personel : {0}\n".format(allArgsAre[1]))
    logTxtFile.filePrint("Tarihsel : {0}\n".format(allArgsAre[2]))
    logTxtFile.filePrint('Klasör   : {0}\n\n'.format(os.getcwd()))
    return allArgsAre

def allOthers(allFilesAre):
    """
        all sanity checks...
        are all compulsory (and also optional file if exist) files available ?
        which columns of the files will be used ?
        ...
    """
    # global logTxtFile
    dataFramesAndColumns = list()
    # are files available ?
    for fileNoIs, eachOneFile in enumerate(allFilesAre, start=1):
        pathAndFileName = os.getcwd() + "\\" + eachOneFile
        fileAvailableList = list()
        fileAvailableList = glob.glob(pathAndFileName)
        #if it is available then read it...
        if len(fileAvailableList) == 1:
            #read the file...
            if eachOneFile.split('.')[-1] in ['xlsx', 'csv']:
                #print("file extension is : {0}".format(eachOneFile.split('.')[-1]))
                if eachOneFile.split('.')[-1] == 'xlsx': #Excel...
                    dfTemp = pd.read_excel(pathAndFileName)
                else: #csv
                    # Code page 1254 Windows-Turkish...
                    dfTemp = pd.read_csv(pathAndFileName, sep = ';', encoding = 'cp1254')
                print("{0} isimli dosya okundu...".format(pathAndFileName))
                logTxtFile.filePrint("{0} isimli dosya okundu...\n".format(pathAndFileName))
                if fileNoIs == 3:
                    print("Atamalarda, geçmiş atamaları gösterir dosya da baz alınacaktır...")
                    logTxtFile.filePrint("\nAtamalarda, geçmiş atamaları gösterir dosya da baz alınacaktır...\n")
            else:
                print("Dosya uzantısı xlsx | csv olmalıdır !")
                print("Program sonlandırıldı !")
                sys.exit(-10001)
            if fileNoIs == 2: # is there at least one person in the personnel dataframe ?
                if len(dfTemp.index) < 1:
                    print("Personel dosyasında hiç personel ismi bulunmuyor !!!")
                    print("Program sonlandırılıyor...")
                    sys.exit(-800)
            elif fileNoIs == 1: # is there at least one row in the input dataframe ?
                if len(dfTemp.index) < 1:
                    print("Girdi Atama dosyasında hiç kayıt bulunmuyor !!!")
                    print("Program sonlandırılıyor...")
                    sys.exit(-851)
            else:
                # global PastAssignmentsAvailable
                if not PastAssignmentsAvailable:
                    if len(dfTemp.index) < 1:
                        print("Geçmiş atama dosyasında hiç kayıt bulunmuyor !!!")
                        print("Program sonlandırılıyor...")
                        sys.exit(-951)
                else:
                    pass
        else:
            if fileNoIs == 3: #if allFilesAre[2] is None... (past assignments not available...)
                # global PastAssignmentsAvailable
                if not PastAssignmentsAvailable:
                    print("Dikkat  : Geçmiş atamaları gösterir dosya verilmedi...")
                    print("Atamalar eldeki dosyalar kulanılarak yapılacaktır !")
                    logTxtFile.filePrint("Dikkat  : Geçmiş atamaları gösterir dosya verilmedi...\n")
                    logTxtFile.filePrint("Atamalar eldeki dosyalar kulanılarak yapılacaktır !\n")
                else:
                    print("Geçmiş atamaları gösterir dosya klasörde mevcut değildir !")
                    print("Program sonlandırıldı !")
                    sys.exit(-11111)
            else:
                print("Zorunlu dosya verilmemiştir !")
                print("Program sonlandırıldı !")
                sys.exit(-10101)
        # File reading is finished. Now, check the columns of the file...
        colNames = list(dfTemp.columns)
        ### !!!
        if fileNoIs == 1: #just for the main input file...
            if 'PERSONEL' in colNames:
                print("Atanacak kayıtların olduğu girdi dosyası PERSONEL isminde bir kolon içeremez !")
                print("PERSONEL isimli kolonu silip yeniden çalıştırınız !")
                print("Program sonlandırıldı...")
                sys.exit(-999)
            else:    
                pass # do nothing !
                """
                colNames.insert(0, 'PERSONEL')
                #### insert column called PERSONEL into the dataframe !!!
                operObject = mymodule.SomeFuncs()
                operObject.insertColumn(dfTemp.shape[1], "---", "PERSONEL", dfTemp)
                """
        #read column names...
        while True:
            for inc, dispFiles in enumerate(colNames, start=1):
                #print("| ", inc, ":", dispFiles, "\t| ", end='')
                print("{0:3} : {1}".format(inc, dispFiles))
            try:
                isValid = True
                tempList = list()
                if fileNoIs == 2: #just for personnel file...
                    columnNameIs = input("Personel dosyası, personel isimlerini içeren kolon ? (Kolon No belirtiniz !)")
                elif fileNoIs == 1: #just for input file...
                    columnNameIs = input("Girdi Atama dosyası, anahtar kolonu ? (Kolon No belirtiniz !)")
                else: # past assignments...
                    columnNameIs = input("Anahtar/Personel kolonları ? (Kolon No belirtiniz ! (aralarına / koyarak))")
                    columnNameIs = columnNameIs.split('/', 1)
                    # if this code block is used, we get something different !!!
                    # every digit of your console input is processed !!!
                    """
                    for justOne in columnNameIs:
                        print("\n\nkuna: {0}\n\n".format(justOne))
                        tempList.append(int(justOne))
                        if not (int(justOne) in range(1, len(colNames)+1)):
                            print("Geçerli değerler : 1 ... ", len(colNames))
                            isValid = False
                            break
                    """ 
                if ((fileNoIs == 1) or (fileNoIs == 2)):
                    if not (int(columnNameIs) in range(1, len(colNames)+1)):
                        print("Geçerli değerler : 1 ... ", len(colNames))
                        isValid = False
                        continue
                    else:    
                        tempList.append(int(columnNameIs))
                        break
                else:
                    for justOne in columnNameIs:
                        try:
                            tempList.append(int(justOne))
                        except:
                            print("Geçerli değerler üzerinden bir giriş yapınız !")
                            # isValid = False
                            # continue
                        else:    
                            if not (int(justOne) in range(1, len(colNames)+1)):
                                print("Geçerli değerler : 1 ... ", len(colNames))
                                isValid = False
                                continue
            except ValueError:
                print("Tamsayı girilmelidir !")
            except:
                print("Geçerli bir giriş bekleniyor !")
            else:
                if isValid:
                    if fileNoIs == 2: #just for personnel file...
                        if len(tempList) == 1:
                            break
                        else:
                            print("Personel isimlerini içeren tek kolon bildirilmelidir !")
                            continue
                    elif fileNoIs == 1: #just for input file...
                        if len(tempList) == 1:
                            break
                        else:
                            print("Anahtar kolon olarak yalnızca bir kolon bildirilmelidir !")
                            continue
                    else: # just for past asignments...
                        if len(tempList) == 2:
                            if tempList[0] != tempList[1]:
                                break
                            else:    
                                print("Anahtar ve Personel kolonları aynı kolon olamaz !")
                                continue
                        else:
                            print("Anahtar/Personel olmak üzere yalnızca iki kolon bildirilmelidir !")
                            continue    
                else:
                    continue
        ### !!!
        # assign as a tuple...
        if fileNoIs == 3: # just for past assignments...
            ### !!!
            dataFramesAndColumns.append(    (  dfTemp, (   colNames[tempList[0]-1], colNames[tempList[1]-1]  )   )   )

            logTxtFile.filePrint("Geçmiş atamalar dosyası için Anahtar  Kolon : {0}\n".format(colNames[tempList[0]-1]))
            logTxtFile.filePrint("Geçmiş atamalar dosyası için Personel Kolon : {0}\n\n".format(colNames[tempList[1]-1]))
        else: # both input and personnel...
            ### !!!
            dataFramesAndColumns.append(   (   dfTemp, colNames[tempList[0]-1]   )   )

            if fileNoIs == 1: # just for input dataframe
                logTxtFile.filePrint("Girdi atama dosyası için Anahtar  Kolon    : {0}\n\n".format(colNames[tempList[0]-1]))
            else:  # just for personnel dataframe
                logTxtFile.filePrint("Personel girdi dosyası için Personel Kolon : {0}\n\n".format(colNames[tempList[0]-1]))
        headOfProg()
        """
        # all approach below works...
        # one...
        # ******
        print("\n*******\n")
        print("Column(s)  : \n", selectedColumns[fileNoIs-1])
        print("Dataframe  : \n", dfTempList[fileNoIs-1].head(1).to_string())
        if fileNoIs == 2: #just for personnel file...
            print("Value(s)   : \n", dfTempList[fileNoIs-1][selectedColumns[fileNoIs-1]].head(1).to_string())
        else:    
            print("Value(s)   : \n", dfTempList[fileNoIs-1][list(selectedColumns[fileNoIs-1])].head(1).to_string())
        print("\n*******\n")
        _ = input("tuşla")
        # two...
        # ******
        if fileNoIs == 2: #just for personnel file...
            xyz.append(dfTempList[fileNoIs-1], selectedColumns[fileNoIs-1])
        else:
            xyz.append(dfTempList[fileNoIs-1], list(selectedColumns[fileNoIs-1])[0], list(selectedColumns[fileNoIs-1])[1])
        # three...
        # ********
        print("\n*******\n")
        for incOfOutside in range(0, len(allFilesAre)):
            print("Dataframe: ", dfTempList[incOfOutside].head(1))
            for incOfInside in range(0, 1 if incOfOutside == 1 else 2):
                print("Columns  : ", selectedColumns[incOfInside], "\n")
            _ = input("tuşla")
        """        
    """
    ### display...    
    for i in [0, 1, 2]:
        if i == 0:
            print("Girdi:")
            print("Anahtar :>{0}<".format(   dfTempList[i].loc[ 0, list(selectedColumns[i])[0] ]   ))
            print("Personel:>{0}<".format(   dfTempList[i].loc[ 0, list(selectedColumns[i])[1] ]   ))
        elif i == 1:
            print("Personel:")
            print("Personel:>{0}<".format(   dfTempList[i].loc[ 0, selectedColumns[i] ]   ))
        else:
            print("Geçmiş:")
            print("Anahtar :>{0}<".format(   dfTempList[i].loc[ 0, list(selectedColumns[i])[0] ]   ))
            print("Personel:>{0}<".format(   dfTempList[i].loc[ 0, list(selectedColumns[i])[1] ]   ))
    """     
    return dataFramesAndColumns
    
def displayPrinciples():
    """
        just for checking something...
    """
    # print("\n\n ... {0} ...\n\n".format(len(allSequences)))
    print("\nGirdi:")
    print("Anahtar :>{0}<".format(   allSequences[0][0].loc[ 0, allSequences[0][1] ]   ))

    print("\nPersonel:")
    print("Personel:>{0}<".format(   allSequences[1][0].loc[ 0, allSequences[1][1] ]   ))
    
    print("\nGeçmiş:")
    print("Anahtar :>{0}<".format(   allSequences[2][0].loc[ 0, allSequences[2][1][0] ]   ))
    print("Personel:>{0}<".format(   allSequences[2][0].loc[ 0, allSequences[2][1][1] ]   ))

def writeAllToText():
    """
        write all process information to the text file...
        may be examine later...
    """
    # change folder and create a text file...
    logTxtFile = mymodule.WritingTextFile(os.getcwd() + '\\')
    # def initLogFile():
    # can not concat this form : (r"d:\IKFA\Personel_Tahsis\" + nameOfFile)
    # logTxtFile = mymodule.WritingTextFile(pathOfProg)
    if logTxtFile.fileExist():
        # file is not needed to delete. Because,
        # if file is reopened in read-write mode, all is gone...
        # fileDelete(path, nameOfFile)
        pass # reevaluate if necessary...
    else:
        logTxtFile.fileCreate()
        logTxtFile.fileClose()

    logTxtFile.fileOpen()
    logTxtFile.filePrint("\nMurat BAYRAV, Rabbi Yüce ALLAH'a şükreder !!!\n")
    logTxtFile.filePrint("Text log dosya oluşturuldu...\n")
    # print("İşlemler ile ilgili detaylar, " + logTxtFile.path + "\\" + logTxtFile.nameOfFile + "\nisimli dosyaya yazılacaktır...")    
    print("İşlemler ile ilgili detaylar, bu klasör içinde " + logTxtFile.nameOfFile + " isimli dosyaya yazılacaktır...")    
    logTxtFile.filePrint("\nİşlem başladı... \n\n")
    print("\nİşlem başladı... \n")
    return logTxtFile
    
def processGroup():
    """
    groupby of the input dataframe and then count items...
    """
    """
        print("\n", allSequences[0][0].head(1).to_string()) # input dataframe...
        print("\n", allSequences[0][1][0])
        print("\n", allSequences[0][0][ allSequences[0][1][0] ].head(1).to_string())

        print("\n", allSequences[2][0].head(1).to_string()) # past info dataframe...
        print("\n", allSequences[2][1][0])
        print("\n", allSequences[2][0][ allSequences[2][1][0],  allSequences[2][1][1]].head(1).to_string()) # key, personnel
        
        print("\n", allSequences[1][0].head(1).to_string()) # personnel info dataframe...
        print("\n", allSequences[1][1][0])
        print("\n", allSequences[1][0][ allSequences[1][1][0] ].head(1).to_string()) # just personnel
    """
    ### Take these codes below... (5) ###
    groupedCheck = allSequences[0][0].groupby(allSequences[0][1])
    itemCountIs = 0
    totValIs = 0
    # print("Times x Value = Total")
    # print("Adet  x Değer = Toplam")
    print("\nAdet  x Değer")
    logTxtFile.filePrint("\nAdet  x Değer\n")
    # Write into the text log file...
    for keyIs,valueIs in groupedCheck.groups.items():
        # print("\n", keyIs, valueIs, len(valueIs), end = ' ')
        itemCountIs += 1
        # totValIs += valueIs.size*keyIs
        totValIs += len(valueIs)
        # print("{:>3d}   x{1}    ={:>3d}".format(valueIs.size, keyIs, valueIs.size*keyIs))
        """
            print("{:>3d}   x {:<s}".format(len(valueIs), keyIs))
        """
        logTxtFile.filePrint("{:>3d}   x {:<s}\n".format(len(valueIs), keyIs))
    # Write into the text log file...
    # print("\n  Total : {0} element and {1} assignments...".format(itemCount, totVal))        
    print("\nToplam {0} farklı değer ve {1} atama yapılacak...\n".format(itemCountIs, totValIs))
    print("Detay için bu klasördeki text dosyaya bakınız...\n")
    # Write into the text log file...
    logTxtFile.filePrint("\nToplam {0} farklı değer ve {1} atama yapılacak...\n\n".format(itemCountIs, totValIs))
    
def offerPersonel():
    """
        examine past assignments and then offer personnel based on past assignments, if suitable...
    """
    """
        print("\n", allSequences[0][0].head(1).to_string()) # input dataframe...
        print("\n", allSequences[0][1][0])
        print("\n", allSequences[0][0][ allSequences[0][1][0] ].head(1).to_string())

        print("\n", allSequences[2][0].head(1).to_string()) # past info dataframe...
        print("\n", allSequences[2][1][0])
        print("\n", allSequences[2][0][ allSequences[2][1][0],  allSequences[2][1][1]].head(1).to_string()) # key, personnel
        
        print("\n", allSequences[1][0].head(1).to_string()) # personnel info dataframe...
        print("\n", allSequences[1][1])
        print("\n", allSequences[1][0][ allSequences[1][1][0] ].head(1).to_string()) # just personnel
    """
    tempDataframe = allSequences[1][0]
    tempDataframe.reset_index(inplace = True)
    personelList = tempDataframe[allSequences[1][1]].tolist()
    # allSequences is a list and it has tuples. Tuples are immutable !
    # so, change list values...
    # reassign personnel file
    allSequences[1] = (allSequences[1][0], (allSequences[1][1][0], allSequences[1][1][1]))
    # carry the following statement when reading the file !!!
    # reassign past assignments
    allSequences[2] = (allSequences[2][0][[ allSequences[2][1][0], allSequences[2][1][1] ]], (allSequences[2][1][0], allSequences[2][1][1]))
    dfIKFAGrouped = allSequences[2][0].groupby([allSequences[2][1][0], allSequences[2][1][1]])
    # key is KEY and personnel is PERSONEL
    dfGecmis = dfIKFAGrouped.size().reset_index().rename(columns = {allSequences[2][1][0]:'KEY', allSequences[2][1][1]:'PERSONEL', 0:'ADET'})
    dfGecmisPivoted = dfGecmis.pivot(index = 'KEY', columns = 'PERSONEL', values = 'ADET')
    # There may be more than one PERSONEL to have been assigned !!!
    # Thus, who is the most assigned ?!
    # This PERSONEL will be optimum, if She/He is in the list named personelList !!!
    # ******************************************************************************
    # There should not be the similar personnel names in the personnel file ...
    # for instance, "ALİ" and "ALİ " actually is the same...
    # data editing should have been before processing...
    print("\n\n")
    print("{0}".format(dfGecmisPivoted.head().to_string()))
    _ = input("tuşla...")
    newCol = list()
    for index, row in dfGecmisPivoted.iterrows(): # Dataframe loop...
        oneSerie = row[index : ] # One row...
        dfProcess = oneSerie.to_frame()
        dfProcess.dropna(inplace = True)
        dfProcess.reset_index(inplace = True)
        # Sort by assigments in order to obtain max...
        dfProcess.sort_values(by = dfProcess.columns[1], axis = 0, ascending = False, inplace = True)
        dfProcess.reset_index(inplace = True)
        # Is this PERSONEL found in the list named personelList ?!
        isFound = False
        for inc in range(0, len(dfProcess.index)): # dfProcess.columns
            # if this PERSONEL is not found, move on to the next...
            for pers in personelList:
                if pers == dfProcess.loc[inc, 'PERSONEL']:
                    isFound = True
                    break
            if isFound:
                break
        if not isFound: # No one exists exactly...
            pers = " " # Assign blank...
        ### Important ###
        # if an empty string is (not) assigned to the pers...
        # then, groupby mechanism does not count NaNs in size() !
        #################
        newCol.append(pers)
    dfGecmisPivoted['ONERILEN'] = newCol

    gg = 1
    for kk in dfGecmisPivoted.columns:
        print("column no {0} : {1}".format(gg, kk))
        gg += 1
    print("\n")    
    print("\nofferPersonel :\n{0}\n".format(dfGecmisPivoted.head(1).to_string()))
    
    if PastAssignmentsAvailable:
        print("Geçmiş atamalar kullanılarak, her anahtar alan için, ideal personel önerilmiştir...")
        logTxtFile.filePrint("\nGeçmiş atamalar kullanılarak, her anahtar alan için, ideal personel önerilmiştir...")
        
    return dfGecmisPivoted
    
def examinePast(retDataframe):
    """
        examine past assignments and then offer personnel based on past assignments, if suitable...
    """
    """
        print("\n", allSequences[0][0].head(1).to_string()) # input dataframe...
        print("\n", allSequences[0][1][0])
        print("\n", allSequences[0][0][ allSequences[0][1][0] ].head(1).to_string())

        print("\n", allSequences[2][0].head(1).to_string()) # past info dataframe...
        print("\n", allSequences[2][1][0])
        print("\n", allSequences[2][0][ allSequences[2][1][0],  allSequences[2][1][1]].head(1).to_string()) # key, personnel
        
        print("\n", allSequences[1][0].head(1).to_string()) # personnel info dataframe...
        print("\n", allSequences[1][1])
        print("\n", allSequences[1][0][ allSequences[1][1][0] ].head(1).to_string()) # just personnel
    """
    # if PastAssignmentsAvailable:
    retDataframe.reset_index(inplace = True)
    retDataframe = retDataframe[['KEY', 'ONERILEN']]
    dfTemp = pd.merge(left = allSequences[0][0], right = retDataframe, how = 'left', left_on = allSequences[0][1], right_on = 'KEY')
    dfTemp.drop('KEY', axis = 1, inplace = True)
    # dfTemp.rename(columns = {'ONERILEN' : 'PERSONEL'}, axis = 1, inplace = True)
    ### !!! double ???
    dfTemp = dfTemp.rename(columns = {'ONERILEN' : 'PERSONEL'}) ### !!!
    ### Important ###
    # if NaN is existed for PERSONEL, replace with blank...
    # Otherwise groupby does not count NaNs !!!
    #################
    dfTemp['PERSONEL'].fillna(value = " ", inplace = True)
    ### !!!
    # allSequences is a list and it has tuples. Tuples are immutable !
    # so, change list values...
    # allSequences[0][0] = dfTemp
    allSequences[0] = ( dfTemp, ( allSequences[0][1] ) )

    # if it exists ????????
    tempData = dfTemp.reset_index()
    """
    print("*******\n")
    print("\n{0}\n".format(tempData.columns))
    print(tempData.PERSONEL.head(3).to_string())
    print("*******\n")
    """
    tempList = tempData['PERSONEL'].tolist() # !!!
    tempSet = set(tempList) # just unique values for personnel...
    tempList = list(tempSet)
    tempList.sort()
    # tempList.insert(1, "Murat")
    if len(tempList) == 1 and tempList[0] == ' ':
        print("Geçmiş verilerden hareketle önerilebilecek bir personel bulunmamaktadır !")
        # Write into the text log file...
        logTxtFile.filePrint("Geçmiş verilerden hareketle önerilebilecek bir personel bulunmamaktadır !")
    else:
        if tempList[0] == ' ':
            tempList.pop(0)
        print("İdeal personel geçmiş veriler incelenerek önerildi...")
        print("Önerilen toplam {0} personel : {1}\n".format(len(tempList), tempList))
        # Write into the text log file...
        logTxtFile.filePrint("\nİdeal personel geçmiş veriler incelenerek önerildi...\n")
        logTxtFile.filePrint("Önerilen toplam {0} personel : {1}\n".format(len(tempList), tempList))
"""        
else:
    operObject = mymodule.SomeFuncs()
    operObject.insertColumn(allSequences[0][0].shape[1], "---", "PERSONEL", allSequences[0][0]) #####
    print("Atama dosyasına PERSONEL kolonu eklendi...")
    # Write into the text log file...
    logTxtFile.filePrint("Atama dosyasına PERSONEL kolonu eklendi...\n")
"""

def correctness(df):
    """
        is the column available ?
        it is somewhat a sanity check !
    """
    if df['PERSONEL'] == ' ':
        return 0
    else:
        return df['PLANLANAN']

def alibaba():
    """
    ...set process files and run the algorithm...
    """
    """
    print("\nGirdi:")
    print("Anahtar :>{0}<".format(   allSequences[0][0].loc[ 0, allSequences[0][1] ]   ))

    print("\nPersonel:")
    print("Personel:>{0}<".format(   allSequences[1][0].loc[ 0, allSequences[1][1] ]   ))
    
    print("\nGeçmiş:")
    print("Anahtar :>{0}<".format(   allSequences[2][0].loc[ 0, allSequences[2][1][0] ]   ))
    print("Personel:>{0}<".format(   allSequences[2][0].loc[ 0, allSequences[2][1][1] ]   ))
    """
    ### Take these codes below... (1) ###
    # input file.. (key and per)
    groupAtama = allSequences[0][0].groupby([allSequences[0][1], 'PERSONEL'])
    icmalKume = groupAtama.size().reset_index().rename(columns = {'KEY':'KEY', 'PERSONEL':'PERSONEL', 0:'PLANLANAN'})
    operObject2 = mymodule.SomeFuncs()
    operObject2.insertColumn(icmalKume.shape[1], "", "ATANAN", icmalKume)
    operObject2.insertColumn(icmalKume.shape[1], "", "FARK", icmalKume)
    icmalKume['ATANAN'] = icmalKume.apply(correctness, axis = 1) ### !!!
    icmalKume['FARK'] = icmalKume['PLANLANAN'] - icmalKume['ATANAN']
    print("icmalKume isimli dataframe oluşturuldu...")
    # Write into the text log file...
    logTxtFile.filePrint("\nicmalKume isimli dataframe oluşturuldu...\n")

    ### Take these codes below... (2) ###
    tempOne = icmalKume.groupby(['PERSONEL'])['PERSONEL'].nunique().to_frame()
    tempTwo = tempOne.drop(labels = 'PERSONEL', axis = 1)
    tempTwo.reset_index(inplace = True)
    if tempTwo.loc[0, 'PERSONEL'].strip() == '': #####
        ### !!!
        # tempTwo.drop(index = 0, inplace = True) # beginning from pandas 0.21.0 !!!
        ### !!!
        tempTwo.drop(tempTwo.index[0], inplace = True) # is equivalent ???
    kumeList = list(tempTwo['PERSONEL'])

    # personnel
    tempOne = allSequences[1][0].groupby(['PERSONEL'])['PERSONEL'].nunique().to_frame()
    tempTwo = tempOne.drop(labels = 'PERSONEL', axis = 1)
    tempTwo.reset_index(inplace = True)
    if tempTwo.loc[0, 'PERSONEL'].strip() == '': #####
        ### !!!
        # tempTwo.drop(index = 0, inplace = True) # beginning from pandas 0.21.0 !!!
        ### !!!
        tempTwo.drop(tempTwo.index[0], inplace = True) # is equivalent ???
    personelList = list(tempTwo['PERSONEL'])
    ### !!!
    ### filter list by using the other list...
    farkliPersList = list(filter(lambda x : not (x in kumeList), personelList))

    print("Geçmiş veriden elde edilen personel : {0}".format(kumeList))
    # Write into the text log file...
    logTxtFile.filePrint("Geçmiş veriden elde edilen personel : {0}\n".format(kumeList))
    print("Personel dosyasında bildirilen personel : {0}".format(personelList))
    # Write into the text log file...    
    logTxtFile.filePrint("Personel dosyasında bildirilen personel : {0}\n".format(personelList))
    print("Personel dosyasındakilerden farklı olanlar : {0}".format(farkliPersList))
    # Write into the text log file...
    logTxtFile.filePrint("Personel dosyasındakilerden farklı olanlar : {0}\n".format(farkliPersList))

    atanacakSatir  = icmalKume['PLANLANAN'].sum()
    atanacakPersonel  = len(personelList)
    # avoid zerodivision ?
    # it is supposed to be not necessary : it had been checked !
    if atanacakPersonel == 0:
        atanacakPersonel  = len(allSequences[1][0].index)
    atanacakOrtSatir = atanacakSatir//atanacakPersonel + 1
    esneklik = atanacakPersonel*atanacakOrtSatir - atanacakSatir
    print("\nAtama yapılacak toplam kayıt : {0}".format(atanacakSatir))
    # Write into the text log file...
    logTxtFile.filePrint("\nAtama yapılacak toplam kayıt : {0}\n".format(atanacakSatir))
    print("Görev verilecek toplam personel : {0}".format(atanacakPersonel))
    # Write into the text log file...
    logTxtFile.filePrint("Görev verilecek toplam personel : {0}\n".format(atanacakPersonel))
    print("Personel başına atama yapılacak ortalama kayıt : {0}".format(atanacakOrtSatir))
    # Write into the text log file...
    logTxtFile.filePrint("Personel başına atama yapılacak ortalama kayıt : {0}\n".format(atanacakOrtSatir))
    print("Atamada kullanılacak esneklik : {0}".format(esneklik))
    # Write into the text log file...
    logTxtFile.filePrint("Atamada kullanılacak esneklik : {0}\n".format(esneklik))

    # input file...
    groupPersonel = allSequences[0][0].groupby(['PERSONEL']) # for Personel...
    icmalPersonel = groupPersonel.size().reset_index().rename(columns = {'PERSONEL':'PERSONEL', 0:'ATANAN'})
    if icmalPersonel.loc[0, 'PERSONEL'] == ' ': ####
        ### !!!
        # icmalPersonel.drop(index = 0, inplace = True) # beginning from pandas 0.21.0 !!!
        ### !!!
        icmalPersonel.drop(icmalPersonel.index[0], inplace = True) # is equivalent ???
    ### !!!
    icmalPersonel['PLANLANAN'] = list(atanacakOrtSatir for cycle in range(0, len(icmalPersonel.index))) ### list comprehension...
    icmalPersonel['FARK'] = icmalPersonel['PLANLANAN'] - icmalPersonel['ATANAN']
    print("icmalPersonel isimli dataframe oluşturuldu...")
    # Write into the text log file...
    logTxtFile.filePrint("icmalPersonel isimli dataframe oluşturuldu...")

    ### Take these codes below... (3) ###
    if len(farkliPersList) > 0:
        for incVal in range(0, len(farkliPersList)):
            icmalPersonel = icmalPersonel.append( \
            {'PERSONEL'  : farkliPersList[incVal], \
             'ATANAN'    : 0, \
             'PLANLANAN' : atanacakOrtSatir, \
             'FARK'      : atanacakOrtSatir}, ignore_index=True)
        # convert float64 into int64...
        icmalPersonel['PLANLANAN'] = icmalPersonel['PLANLANAN'].astype(np.int64)    
        icmalPersonel['FARK'] = icmalPersonel['FARK'].astype(np.int64)    
        print("\nPersonel listesine eklenen personel : {0}".format(farkliPersList))
        # Write into the text log file...
        logTxtFile.filePrint("\nPersonel listesine eklenen personel : {0}\n".format(farkliPersList))
    else:
        print("\nPersonel listesine personel eklenmesi gerekmiyor !")
        # Write into the text log file...
        logTxtFile.filePrint("\nPersonel listesine personel eklenmesi gerekmiyor !\n")

    ### Pause the running by asking input from user ? Are you sure to move on to iteration stage ?
    ### Take these codes below... (4) ###
    print("\nAtamalar için iterasyon başladı...\n")
    # Write into the text log file...
    logTxtFile.filePrint("\nAtamalar için iterasyon başladı...\n")

    iterationNum = -1
    while(True):
        iterationNum = iterationNum + 1
        print("Iterasyon No : {0}".format(iterationNum))
        # Write into the text log file...
        logTxtFile.filePrint("Iterasyon No : {0}\n".format(iterationNum))

        icmalPersonel.sort_values('FARK', ascending = False, inplace = True)
        icmalPersonel.reset_index(inplace = True)
        icmalPersonel.drop('index', axis = 1, inplace = True)
        print(icmalPersonel.to_string())
        # Write into the text log file...
        logTxtFile.filePrint("{0}\n".format(icmalPersonel.to_string()))

        icmalKume.sort_values('FARK', ascending = False, inplace = True)
        icmalKume.reset_index(inplace = True)
        icmalKume.drop('index', axis = 1, inplace = True)
        print(icmalKume.to_string())
        # Write into the text log file...
        logTxtFile.filePrint("{0}\n".format(icmalKume.to_string()))

        if icmalKume.loc[icmalKume.index[0], 'FARK'] == 0:
            break

        icmalKume.loc[icmalKume.index[0], 'PERSONEL'] = icmalPersonel.loc[icmalPersonel.index[0], 'PERSONEL']
        icmalPersonel.loc[icmalPersonel.index[0], 'ATANAN'] += icmalKume.loc[icmalKume.index[0], 'PLANLANAN']
        ### !!!
        icmalKume.loc[icmalKume.index[0], 'ATANAN'] = icmalKume.loc[icmalKume.index[0], 'PLANLANAN'] # all records to one personal...

        icmalPersonel.loc[icmalPersonel.index[0], 'FARK'] = icmalPersonel.loc[icmalPersonel.index[0], 'PLANLANAN'] - icmalPersonel.loc[icmalPersonel.index[0], 'ATANAN']
        icmalKume.loc[icmalKume.index[0], 'FARK'] = icmalKume.loc[icmalKume.index[0], 'PLANLANAN'] - icmalKume.loc[icmalKume.index[0], 'ATANAN']
    
    print("\nİterasyon tamamlandı...\n")
    print("\nToplam {0} iterasyon sonra sonuca ulaşıldı...\n".format(iterationNum))
    logTxtFile.filePrint("\nİterasyon tamamlandı...\n")
    logTxtFile.filePrint("\n" + "Toplam " + str(iterationNum) + " iterasyon sonra sonuca ulaşıldı...\n")

    ##### The last operations... #####
    ##################################

    print("\nİşlemler tamamlandı... \n")
    logTxtFile.filePrint("\nİşlemler tamamlandı... \n")
    logTxtFile.filePrint("\nDosya kapatıldı...\n")
    logTxtFile.fileClose()

    # print("İşlemler ile ilgili detaylar, \n" + logTxtFile.path + "\\" + logTxtFile.nameOfFile + "\nisimli dosyaya yazılmıştır...")

def main():
    ### Ctrl+C : keyboard interrupt
    setRelatedOption()
    headOfProg()
    global logTxtFile
    logTxtFile = writeAllToText()
    global allSequences
    allSequences = allOthers(allArgs())
    displayPrinciples()
    processGroup()
    # !!!
    # examine past assignments...
    # offer personnel based on past assignments, if past assigments are available...
    # !!!
    global PastAssignmentsAvailable
    if PastAssignmentsAvailable:
        examinePast(offerPersonel())
    alibaba()        
    
if __name__ == "__main__":
    main()

----------------------------------------
Developer: Murat BAYRAV / 12-2018 ( @ ) 
----------------------------------------
İşin personele dağıtılması...
Küme iş, bütün olarak bir personele,
Tüm işler eksiksiz olarak,
Her küme iş geçmişte o işi yapana,
Eşit iş yükü olacak şekilde...
----------------------------------------
İşlemler ile ilgili detaylar, bu klasör içinde Log_14-05-2019_15-15-25.txt isimli dosyaya yazılacaktır...

İşlem başladı... 


Zorunlu argümanlar verilmelidir !
Program sonlandırıldı...

Programın Kullanımı : __main__.py [-g TARIHSEL] Girdi Personel

Pozisyonel parametreler:
  Girdi                 Tasnif edilecek girdi dosyası (csv | xlsx)
  Personel              Aktif personel dosyası (xlsx)

Opsiyonel parametreler:
  -g TARIHSEL, --Tarihsel TARIHSEL
                        Geçmiş atamalar dosyası (xlsx)



Programın Kullanımı : __main__.py [-g TARIHSEL] Girdi Personel
__main__.py: error: the following arguments are required: Personel


SystemExit: -111

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
